In [33]:
## data processsing
import json
import pandas as pd
import random
import pickle
from sklearn.preprocessing import MultiLabelBinarizer

############################# 
############ file path
filepath = "twitter.json"
filepath2 = "twits_raw_20180725.json"
#############################

In [2]:
def load_json_multiple(segments):
    chunk = ""
    news_list = []
    for segment in segments:
        chunk += segment
        try:
            yield json.loads(chunk)
            chunk = ""
        except ValueError:
            pass
def get_news_list(path):
    news_list = []
    with open(path) as f:
        for parsed_json in load_json_multiple(f):
            news_list.append(parsed_json)
    return news_list
def get_tag_list(twit_list):
    tag_list = []
    for twit in twit_list:
        for tag in twit['tags']:
            if tag not in tag_list and tag != 'NONE':
                tag_list.append(tag)
    return tag_list
def get_tagged_new_list(path):
    twit_list = get_news_list(path)
    tag_list = get_tag_list(twit_list)

    news_list_G9 = [ item for item in twit_list 
                    if bool(set(tag_list) & set(item['tags']))]

    news_list_none = [item for item in twit_list if item['tags'] == ['NONE']]
    return news_list_G9, news_list_none

tagged_news, none_tagged_news = get_tagged_new_list(filepath)
tagged_news2, none_tagged_news2 = get_tagged_new_list(filepath2)

In [35]:
sample_twits_list = tagged_news2.copy()
random.shuffle(sample_twits_list)
X = ["author: " + twit["author"] +" \n " + twit['text'] for twit in sample_twits_list]
y_raw = [twit["tags"] for twit in sample_twits_list]
mb = MultiLabelBinarizer()
y=mb.fit_transform(y_raw)

print(len(y_raw))
print(y_raw[0:10])
print(y[0:10])

def bin2dec(bin_list):
    int_num = 0
    for i, digit in enumerate(bin_list):
        int_num = int_num + digit*(2**i)
    return int_num

y_int = []
for item in y:
    y_int.append(bin2dec(item))
print(y_int[0:10])

66449
[['JPY', 'EUR'], ['AUD', 'USD'], ['USD'], ['USD'], ['EUR', 'GBP'], ['AUD', 'EUR'], ['USD'], ['USD', 'CNY'], ['USD', 'CNY'], ['USD', 'CAD']]
[[0 0 0 0 1 0 1 0 0]
 [1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 1 0 0 0]
 [1 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 0 1]
 [0 0 0 1 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 1]]
[80, 257, 256, 256, 48, 17, 256, 264, 264, 258]


In [38]:
num_test = int(0.075*len(X))
num_val = 0
X_train = X[num_test:]
X_val = X[:num_val]
X_test = X[num_val:num_val+num_test]
# y_train = y[:num_train]
# y_test = y[num_train:]
y_train = y_int[num_test:]
y_val = y_int[:num_val]
y_test = y_int[num_val:num_val+num_test]

print(len(set(y_int)))
print(len(set(y_train)))
print(len(set(y_val)))
print(len(set(y_test)))
print(len(X_train), len(y_train))

126
122
0
69
61466 61466


In [21]:
len(X)

66449

# model and training

In [22]:
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import lightgbm as lgbm
import time

In [29]:
# lightgbm
t = time.time()
static_params = {
                    'n_estimators': 100,
                    'task': 'train',
                    'boosting_type': 'gbrt',
                    'objective': 'multiclass',
                    'metric': 'multi_logloss',
                    'learning_rate': 0.05,
                    'num_leaves': 31, 
                    'n_jobs': 4, 
                    'reg_alpha': 0.025,
#                     'num_iterations': 200,
#                     'max_bin': 62
                }
print('Start training...')
# Training Support Vector Machines - SVM and calculating its performance
text_clf_lgbm = Pipeline([('vect', CountVectorizer()), 
                         ('tfidf', TfidfTransformer()),
                         ('lgbm', lgbm.LGBMClassifier(
                             **static_params
                         ))
                       ])
text_clf_lgbm = text_clf_lgbm.fit(X_train, y_train)

import pickle
filename = 'multilabel_model_20180725.sav'
pickle.dump(text_clf_lgbm, open(filename, 'wb'))
score = text_clf_lgbm.score(X_test, y_test)
print(score)
print('Start completes...', (time.time() - t)//60)

Start training...
0.9602649006622517
Start completes... 1.0


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
# Random Forest
text_clf_rf = Pipeline([('vect', CountVectorizer()), 
                         ('tfidf', TfidfTransformer()),
                         ('rf', RandomForestClassifier(n_estimators=100, random_state=42, max_depth=90))])
text_clf_rf = text_clf_rf.fit(X_train, y_train)
predicted_rf = text_clf_rf.predict(X_test)
score = text_clf_rf.score(X_test, y_test)
print(score)
filename = 'model/multilabel_model_rf_20180725.sav'
pickle.dump(text_clf_rf, open(filename, 'wb'))

In [30]:
## MLPClassifier
from sklearn.neural_network import MLPClassifier
text_clf_rf = Pipeline([('vect', CountVectorizer()), 
                         ('tfidf', TfidfTransformer()),
                         ('mlp', MLPClassifier(max_iter = 1, early_stopping=True, random_state=42))])
text_clf_rf = text_clf_rf.fit(X_train, y_train)
predicted_rf = text_clf_rf.predict(X_test)
filename = 'model/multilabel_model_mlp_20180725.sav'
pickle.dump(text_clf_rf, open(filename, 'wb'))
score = text_clf_rf.score(X_test, y_test)
print(score)

In [43]:
## voting
t = time.time()
static_params = {
                    'n_estimators': 100,
                    'task': 'train',
                    'boosting_type': 'gbrt',
                    'objective': 'multiclass',
                    'metric': 'multi_logloss',
                    'learning_rate': 0.05,
                    'num_leaves': 31, 
                    'n_jobs': 4, 
                    'reg_alpha': 0.075,
#                     'num_iterations': 200,
#                     'max_bin': 62
                }

mlp_clf = MLPClassifier(max_iter = 5, early_stopping = True, random_state = 42)
# rf_clf = RandomForestClassifier(n_estimators = 100, random_state = 42, max_depth = 90)
lgbm_clf = lgbm.LGBMClassifier(**static_params)
# v_clf = VotingClassifier(estimators=[('mlp', mlp_clf), ('rf', rf_clf), ('lgbm', lgbm_clf)], voting='soft')
text_clf_pl = Pipeline([
                         ('vect', CountVectorizer()), 
                         ('tfidf', TfidfTransformer()),
                         ('v_clf',  VotingClassifier(
                             estimators=[('mlp', mlp_clf), ('lgbm', lgbm_clf)], 
                             voting='soft', weights=[2,14]))
                       ])
text_clf_pl = text_clf_pl.fit(X_train, y_train)
print("training time is ", (time.time()-t)//60)
filename = 'multilabel_model_vote_20180725.sav'
pickle.dump(text_clf_pl, open(filename, 'wb'))
score = text_clf_pl.score(X_test, y_test)
print(score)

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


training time is  7.0
0.9634758177804535


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [40]:
score = text_clf_pl.score(X_test, y_test)
print(score)

0.9624724061810155


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# Error analysis

In [ ]:
## make the results back to binary
y_pred = predicted_rf
def dec2bin(num_list):
    bin_list = []
    for int_num in num_list:
        bin_sub_list = []
        for i in range(9):
    #         print(int_num)
            digit = int_num % 2
            int_num = int(int_num//2)
            bin_sub_list.append(digit)
        bin_list.append(bin_sub_list)
    return bin_list
data_ = {'twit': X_test, 
         'label': mb.inverse_transform(np.asarray(dec2bin(y_test))),
         'label_pred': mb.inverse_transform(np.asarray(dec2bin(y_pred))),
         'correct': y_pred == y_test}
df = pd.DataFrame(data=data_)
df.head()
df.to_csv('data/results.csv')

# saving the model

